#**პრაქტიკული დავალება #5**
## **ვადა (deadline): 21 ივნისი 9:00AM**

- თითოეული სავარჯიშო ფასდება შესაბამისი ქულით, რაც ჯამში შეადგენს **7 ქულას**

**პრობლემის აღწერა:**
ავიაკომპანიები ცდილობენ გააუმჯობესონ მომხმარებელთა კმაყოფილება, რათა შეინარჩუნონ კლიენტები და გაზარდონ ბრენდის ერთგულება. ათასობით მომხმარებლის ყოველდღიური ფრენის გათვალისწინებით, მნიშვნელოვანია მომხმარებელთა კმაყოფილების განმაპირობებელი ფაქტორების გაგება. ჩვენი მონაცემები შეიცავს ინფორმაციას მომხმარებელთა გამოცდილების სხვადასხვა ასპექტისა და მათი ზოგადი კმაყოფილების დონის შესახებ. ამ მონაცემების ანალიზით, ჩვენ უნდა გამოვავლინოთ მომხმარებელთა კმაყოფილების განმსაზღვრელი კანონზომიერებები.

**მიზანი:**
ავაგოთ მოდელი, რომელიც შეგვაძლებინებს დავაპროგნოზოთ, არის თუ არა მომხმარებელი კმაყოფილი ან უკმაყოფილო მათი ფრენის გამოცდილებასთან დაკავშირებული სხვადასხვა მახასიათებლების საფუძველზე. ეს მოდელი დაეხმარება ავიაკომპანიას პროაქტიულად მართოს ფაქტორები, რომლებიც იწვევენ უკმაყოფილებას და გააუმჯობესოს მომხმარებელთა ზოგადი კმაყოფილება.

**[მონაცემები:](https://drive.google.com/drive/folders/1TxMqaH4zYYQYlbsaw0kcMTXLQYXrgwCL?usp=drive_link)**
  - Airline_customer_satisfaction.csv - მონაცემები სამედიცინო ჩანაწერებიდან.
  - Airline_customer_satisfaction_desc.csv - ველების აღწერა.

**მეთოდი:** ლოგისტიკური რეგრესია

**ᲘᲮᲔᲚᲛᲫᲦᲕᲐᲜᲔᲚᲔᲗ:** [ᲚᲔᲥᲪᲘᲐ 10: ᲚᲝᲒᲘᲡᲢᲘᲙᲣᲠᲘ ᲠᲔᲒᲠᲔᲡᲘᲐ](https://colab.research.google.com/drive/1iAfgAsJCgeQlcMpu6SWfciPLkYrxKcMV?usp=sharing)

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

## **სავარჯიშო 1:** მონაცემების ჩატვირთვა და ცარიელი მნიშვნელობების მართვა (**1 ქულა**)
  - ჩატვირთეთ მონაცემები pandas-ის გამოყენებით და აჩვენეთ პირველი რამდენიმე მწკრივი.
  - შეამოწმეთ და სათანადოდ მართეთ გამოტოვებული მნიშვნელობები მათი შევსებით (ჩანაცვლებით) ან წაშლით.



In [ ]:
df = pd.read_csv('data/Airline_customer_satisfaction.csv')

In [ ]:
# მონაცემთა ფორმა
df.shape

In [ ]:
# პირველი 5 მწკრივი
df.head()

In [ ]:
# გამოტოვებული მონაცემები მონაცემები
df.isnull().sum()

In [ ]:
# შევავსოთ გამოტოვებული მნიშვნლეობები მედიანით
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].median())

In [ ]:
# დუბლირებული მონაცემები
df.duplicated().sum()

In [ ]:
# მონაცემთა ტიპები
df.dtypes

In [ ]:
# მონაცემთა აღწერა
df.describe()

## **სავარჯიშო 2:** კატეგორიული ცვლადების კოდირება (**1 ქულა**)
  - გარდაქმენით 'satisfaction' სვეტი ბინარულ რიცხვით ველად - 'dissatisfied'=0-ზე და 'satisfied'=1.
  - გამოიყენეთ one-hot კოდირება სხვა კატეგორიულ ველებზე.



In [ ]:
# დავაკონვერტიროთ satisfaction სვეტი ბინალურ ველად
df['satisfaction'] = df['satisfaction'].map({'dissatisfied': 0, 'satisfied': 1})

# კატეგორიული ველების one-hot კოდირება
categorical_columns = ['Customer Type', 'Type of Travel', 'Class']
df = pd.get_dummies(df, columns=categorical_columns)

## **სავარჯიშო 3:** რიცხვითი ცვლადები ნორმალიზება (**1 ქულა**)
  - განსაზღვრეთ რიცხვითი სვეტები, რომლებიც საჭიროებს ნორმალიზება/სტანდარტიზებას.
  - გამოიყენეთ StandardScaler ამ სვეტებზე.



In [ ]:
# სვეტები რომლებიც საჭიროებს ნორმალიზებას
columns = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']

# გამოვიყენოთ StandartScaler
scaler = StandardScaler()
df[columns] = scaler.fit_transform(df[columns])

## **სავარჯიშო 4:** მონაცემების დაყოფა (**1 ქულა**)
- განსაზღვრეთ მახასიათებლების ნაკრები (X) და სამიზნე ცვლადი (y).
- გაყავით მონაცემები საწვრთნელ და სატესტო ნაწილებად 80/20 თანაფარდობით.



In [ ]:
# განვსაზღვროთ დამოკიდებული და დამოუკიდებელი ცვლადები
X = df.drop('satisfaction', axis=1)
y = df['satisfaction']

# გავყოთ მონაცემები სატესტო და სატრენინგო ნაწილებად
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# სატესტო და სატრენინგო მონაცემების ზომა
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## **სავარჯიშო 5:** მოდელის გაწვრთნა და პროგნოზი (**1 ქულა**)
  - გაწვრთენით ლოგისტიკური რეგრესიის მოდელი საწვრთნელი მონაცემებით.
  - გააკეთეთ პროგნოზები გაწვრთნილი მოდელის გამოყენებით სატესტო მონაცემებზე.

In [ ]:
# მოდელის გაწრთვნა
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# სატესტო სეტზე მოდელის პროგნოზირება
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]

## **სავარჯიშო 6:** მოდელის შეფასება (**1 ქულა**)
გამოთვალეთ მოდელის საზომები და ახსენით, რას მეტყველებს მოდელის ხარისხზე:
  - სიზუსტე (Accuracy)
  - სიცხადე (Precision)
  - მგრძნობელობა (Recall)
  - F1 ქულა (F1 score)
  - ROC AUC
  - ჯინის კოეფიციენტი



In [ ]:
# მეტრიკების გამოთვლა
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)
gini_coefficient = 2 * roc_auc - 1


## **სავარჯიშო 7:** შედეგების ვიზუალიზება (**1 ქულა**)
  - გრაფიკულად გამოსახეთ აღრევის მატრიცა (confusion matrix)
  - სვეტოვანი დიაგრამით გრაფიკულად გამოსახეთ მე-6 სავარჯიშოში დათვლილი საზომები.
  - დახაზეთ ROC მრუდი და მიუთითეთ AUC მნიშვნელობა.
  - შექმენით სიმკვრივის გრაფიკები პროგნოზირებული ალბათობებისთვის, დაყოფილი ფაქტობრივი კლასის მიხედვით (კმაყოფილი და უკმაყოფილო).


In [ ]:
# პლოტი
fig, axs = plt.subplots(2, 2, figsize=(15, 12))

# გადაწყვეტილების მიჯნა
threshold = 0.5

# აღრეუულობის მატრიცა
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', cbar=True,
            xticklabels=['კმაყოფილი', 'უკმაყოფილო'],
            yticklabels=['კმაყოფილი', 'უკმაყოფილო'], ax=axs[0, 0])
axs[0, 0].set_xlabel('ნაპროგნოზები მდგომარეობა')
axs[0, 0].set_ylabel('ნამდვილი მდგომარეობა')
axs[0, 0].set_title('აღრეულობის მატრიცა')

# მეტრიკების bar ჩარტი
metrics = ['სიზუსტე', 'სიცხადე', 'ამოცნობა', 'F1 ქულა', 'ROC AUC', 'ჯინი']
values = [accuracy, precision, recall, f1, roc_auc, gini_coefficient]

bars = axs[0, 1].bar(metrics, values, color=['blue', 'green', 'red', 'purple', 'orange', 'cyan'])
axs[0, 1].set_ylim(0, 1)
axs[0, 1].set_xlabel('მეტრიკა')
axs[0, 1].set_ylabel('მნიშვნელობა')
axs[0, 1].set_title('კლასიფიკაციის ხარისხის მეტრიკები')

# ყოველ სვეტზე მნიშვნელობის დამატება
for bar in bars:
    yval = bar.get_height()
    axs[0, 1].text(bar.get_x() + bar.get_width() / 2, yval + 0.01, round(yval, 2), ha='center', va='bottom')

# ROC მრუდი
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
axs[1, 0].plot(fpr, tpr, label=f'ROC მრუდი (მრუდქვეშა სივრცე = {roc_auc:.2f})')
axs[1, 0].plot([0, 1], [0, 1], 'k--')
axs[1, 0].set_xlabel('ცრუ დადებითი წილი')
axs[1, 0].set_ylabel('ნამდვილი დადებითი წილი')
axs[1, 0].set_title('ROC მრუდი')
axs[1, 0].legend(loc='best')

# ხდომილებათა ალბათობის სიმკვრივის ფუნქციები
clipped_probs = np.clip(y_pred_prob, 0, 1)
satisfaction_probs = clipped_probs[y_test == 1]
non_satisfaction_probs = clipped_probs[y_test == 0]

sns.kdeplot(satisfaction_probs, label='კმაყოფილი', color='green', fill=True, ax=axs[1, 1])
sns.kdeplot(non_satisfaction_probs, label='უკმაყოფილო', color='red', fill=True, ax=axs[1, 1])

# მრუდების ქვეშ სივრცის გაფერადება
satisfaction_kde = sns.kdeplot(satisfaction_probs, color='green', fill=True, ax=axs[1, 1]).get_lines()
non_satisfaction_kde = sns.kdeplot(non_satisfaction_probs, color='red', fill=True, ax=axs[1, 1]).get_lines()

if satisfaction_kde and non_satisfaction_kde:
    satisfaction_x, satisfaction_y = satisfaction_kde[0].get_data()
    non_satisfaction_x, non_satisfaction_y = non_satisfaction_kde[0].get_data()

    axs[1, 1].fill_between(satisfaction_x, satisfaction_y, where=(satisfaction_x > threshold) & (satisfaction_y >= non_satisfaction_y), color='green', alpha=0.85, label='ნამდვილი კმაყოფილი')
    axs[1, 1].fill_between(non_satisfaction_x, non_satisfaction_y, where=(non_satisfaction_x > threshold) & (non_satisfaction_y <= satisfaction_y), color='red', alpha=0.85, label='ცრუ კმაყოფილი')

axs[1, 1].axvline(threshold, color='black', linestyle='--', linewidth=1.5, label='გადაწყვეტილების მიჯნა')
axs[1, 1].set_xlabel('ნაპროგნოზები ალბათობა')
axs[1, 1].set_ylabel('სიმკვრივე')
axs[1, 1].set_xlim(0, 1)
axs[1, 1].set_title('კმაყოფილთა და უკმაყოფილოთა ალბათობების განაწილება')
axs[1, 1].legend()

plt.tight_layout()
plt.show()
